In [ ]:
import os
import sys
sys.path.append("../")

import panel as pn
import altair as alt
from altair import datum
import pandas as pd
# from vega_datasets import data
import datetime as dt

from src.utils.snowflake import read_forecast_meta, read_forecasts
from src.plot.altair import plot_all, plot_decompose

In [ ]:
alt.renderers.enable("default")
pn.extension("vega")
pn.extension('tabulator')
# pn.extension(sizing_mode="stretch_width")

In [ ]:
df_forecast_meta = read_forecast_meta()
df_table = (
    df_forecast_meta.rename(
        columns={
            "vermogen_nominaal": "P_nom",
            "relative_abs_max": "P_max",
            "date_abs_max": "t_max",
        }
    )
    [["boxid", "P_max", "t_max", "vestiging"]]
    .sort_values(by=["P_max", "t_max"], ascending=[False, True])
)
df_table["t_max"] = pd.to_datetime(df_table["t_max"]).dt.date
df_table

In [ ]:
tabulator_formatters = {
    "float": {"type": "progress", "max": 1},
    "bool": {"type": "tickCross"},
}

table = pn.widgets.Tabulator(
    df_table.copy(),
#     formatters=tabulator_formatters,
    show_index=False,
    disabled=True,
    selectable=1,
    height=500,
    width=400,
    selection=[0],
    width_policy="min",
)

vestigingen = list(df_table["vestiging"].unique())
choice_vestiging = pn.widgets.CheckButtonGroup(
    name='regio', 
    value=vestigingen, 
    options=vestigingen,
    width=1100,
)

table.add_filter(choice_vestiging, "vestiging")

def update_tabs(selection):
    # extract boxid
    boxid = table.value["boxid"].iloc[selection].squeeze()
    
    df_total = read_forecasts(boxid=boxid)
    
    plot_total = plot_all(
        df_data=df_total, df_meta=df_forecast_meta[df_forecast_meta["boxid"]==boxid]
    ).properties(width=600, height=400)
    
    plot_min = plot_decompose(df_total[df_total["extreme"]=="min"])
    
    plot_max = plot_decompose(df_total[df_total["extreme"]=="max"])
    
    return pn.Tabs(("general", plot_total), ("breakdown min", plot_min), ("breakdown max", plot_max), width_policy="max")

tabs = pn.bind(update_tabs, selection=table.param.selection)
    
panel = pn.Column(choice_vestiging, pn.Row(table, tabs, min_height=500))
    
pn.serve(panel)